In [3]:
!pip install chromadb sentence_transformers

  Using cached chromadb-1.3.5-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     ---------------------------------------- 42.9/42.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ---------------------------------------- 44.0/44.0 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s et

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.35.0 requires urllib3[socks]<3.0,>=2.5.0, but you have urllib3 2.3.0 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import tarfile

# 압축 해제
print("압축을 해제하는 중...")
with tarfile.open("chroma_db.tar.gz", "r:gz") as tar:
    tar.extractall(path=".")  # 현재 위치에 풀기

print("완료! 'chroma_db' 폴더가 생성되었습니다.")

압축을 해제하는 중...


C:\Users\chris\AppData\Local\Temp\ipykernel_10872\1336457635.py:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=".")  # 현재 위치에 풀기


완료! 'chroma_db' 폴더가 생성되었습니다.


In [1]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="./chroma_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



Using device: cuda
모델 로드 중...
✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


In [10]:
query = "종교와 정치"
query_emb = model.encode(query).tolist()

search_result = collection.query(query_embeddings=[query_emb], n_results=5)

for meatadata in search_result["documents"][0]:
    print(meatadata)
    print()

print(search_result["distances"])

for metadata in search_result["metadatas"][0]:
    print(metadata["patent_id"])
    print(metadata["title"])

.

상기 컴퓨터 프로그램은 운영 체제인, 방법.

방법.

방법.

방법.

[[0.6815519332885742, 0.7060180902481079, 0.7065939903259277, 0.7065939903259277, 0.7065939903259277]]
1020247021287
내장 통증의 치료를 위한 제약 조성물
1020237030113
사이퍼제닉스 기반 생태계 보안 플랫폼들
1020257002260
코딩된 비디오 스트림에서 직사각형 슬라이스 파티셔닝을 시그널링하기 위한 방법
1020257006822
CBF 플래그들의 효율적인 시그널링의 방법
1020257007866
비디오 코딩을 위한 방법 및 장치


In [11]:
# ---------------------------------------------------------
# 4. 검색 함수 정의
# ---------------------------------------------------------
def search_patents(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = model.encode(query_text).tolist()

    # 2) 벡터로 DB 검색
    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k
    )

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")
    
    for i in range(top_k):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i]
        distance = results['distances'][0][i]
        
        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"특허명: {meta.get('title', '제목없음')}")
        print(f"특허ID: {meta.get('patent_id')} (Claim {meta.get('claim_no')})")
        print(f"내용: {doc}") # 100자만 미리보기

# ---------------------------------------------------------
# 5. 실제 검색 테스트
# ---------------------------------------------------------
search_patents("환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법")


🔍 검색어: '환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법'
--- [Rank 1] (거리: 0.2531) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 1)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법으로서,(a) 일련의 EM(electromagnetic) 데이터를 획득하는 단계;(b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하는 단계;(c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하는 단계;(d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 상기 EM 데이터 기반 경로에 매핑함으로써 하나 이상의 변형된 경로를 생성하는 단계; 및(e) 상기 하나 이상의 경로 가설 각각과 관련된 확률에 적어도 부분적으로 기초하여 상기 하나 이상의 변형된 경로로부터 변형된 경로를 선택하고, 그 선택된 변형된 경로에 기초하여 상기 내시경의 팁의 위치를 결정하는 단계를 포함하는, 방법.
--- [Rank 2] (거리: 0.3384) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 17)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 시스템으로서,컴퓨터 실행 가능 명령어를 저장하는 메모리; 및상기 내시경과 통신하고 상기 컴퓨터 실행 가능 명령어를 실행하여, (a) 일련의 EM(electromagnetic) 데이터를 획득하고; (b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하며; (c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하고;  (d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 

In [6]:
def group_results_by_patent(results):
    counts = {}
    metas = results["metadatas"][0]

    for m in metas:
        pat_id = m["patent_id"]
        counts[pat_id] = counts.get(pat_id, 0) + 1

    return counts

In [7]:
# 1. 쿼리 텍스트 정의
query = "편광판을 이용한 액정"

# 2. 데이터를 저장했던 *동일한 모델*로 쿼리를 직접 임베딩
#    (이때 model 변수가 이전에 로드되어 있어야 합니다)
query_emb = model.encode(query).tolist()

# 3. query_texts 대신 query_embeddings 파라미터 사용
results = collection.query(query_embeddings=[query_emb], n_results=10)

# 4. 이후 로직은 동일
patent_hit_map = group_results_by_patent(results)
print("특허별 히트 수:", patent_hit_map)

특허별 히트 수: {'1020247042361': 1, '1020160153262': 1, '1020170024706': 2, '1020247013771': 1, '1020257027309': 1, '1020237001747': 1, '1020250024294': 1, '1020220037297': 1, '1020257023376': 1}


In [10]:
results

{'ids': [['1020247042361_claim8',
   '1020160153262_claim5',
   '1020170024706_claim6',
   '1020247013771_claim4',
   '1020257027309_claim10',
   '1020237001747_claim3',
   '1020250024294_claim2',
   '1020220037297_claim7',
   '1020170024706_claim1',
   '1020257023376_claim7']],
 'embeddings': None,
 'documents': [['제 6 항에 기재한 편광판을 갖는 액정 표시 장치.',
   '액정 셀의 적어도 한쪽의 면에, 제1항 또는 제2항에 기재된 편광판이 배치된 액정 패널.',
   '제1항 내지 제3항 중 어느 한 항에 기재한 편광판 세트와 액정 셀을 구비하고, 상기 액정 셀의 두께가 0.4 mm 이하인 액정 패널.',
   '상기 산성 용액이, 염산, 황산 또는 질산인, 편광판의 제조 방법.',
   '상기 액정 배향막이, 편광 자외선을 조사하여 얻어지는, 액정 배향막.',
   '제1항 또는 상기 편광자층이, 중합성 액정 화합물과 상기 2색성 유기 색소를 함유하는 것을 특징으로 하는 편광판.',
   '편광판용인, 접착제 조성물.',
   '편광판인, 광학 부재.',
   '액정 셀의 시인측에 배치되는 제1 편광판과, 상기 액정 셀의 배면측에 배치되는 제2 편광판을 갖는 편광판 세트로서, 상기 제2 편광판은 반사형 편광 필름을 갖고, 상기 제1 편광판을 80℃에서 4시간 유지했을 때의 흡수축 방향에 있어서의 폭 2 mm당 수축력을 F1로 하고, 투과축 방향에 있어서의 폭 2 mm당 수축력을 F2로 하며, 상기 제2 편광판을 80℃에서 4시간 유지했을 때의 흡수축 방향에 있어서의 폭 2 mm당 수축력을 F3으로 하고, 투과축 방향에 있어서의 폭 2 mm당 수축력을 F4로 했을 때, (F1×F2)/(F3×F4)가 0.5∼